In [1]:
import scipy
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import scipy
import math
import matplotlib.patches as patches
import pylab
import shapely.geometry as geometry
import pylab as plt
from matplotlib.path import Path
import scipy.stats
import shapefile
from matplotlib.path import Path

Code to save the data assimilation estimates and standard deviations.

Read in the ensemble estimates, interpolated to 5 km resolution 

In [2]:
ensemble = xr.open_dataset('indus_interp51823.nc')

Read in the observations and create a mask of the study domain from them. Use them to create a mask of the study domain

In [3]:
obs = xr.open_dataset('obs_wetsnow_v2.nc')
obs_nans = obs.SnowDepth_tavg.max('time')
obs_nans = obs_nans.rename({'north_south': 'lat', 'east_west': 'lon'})
del obs

In [4]:
obs = xr.open_dataset('obs_wetsnow_v2.nc')
obs_mask = ~np.isnan(obs.SnowDepth_tavg.max('time'))
def mask(nc): 
    try: 
        nc = nc.rename({'lon': 'east_west', 'lat': 'north_south'})
    except: 
        nc = nc
    nc = nc.transpose('ensemble', 'time', 'north_south', 'east_west')
    nc = nc.where(obs_mask)
    try:
        nc = nc.__xarray_dataarray_variable__
    except:
        nc = nc
    return nc

Read in the weights

In [7]:
normalized_all = xr.open_dataset('all_assim_weights_wetsnow_30_May18.nc')
normalized_all = normalized_all.__xarray_dataarray_variable__
normalized_all = normalized_all.where(~np.isnan(obs_nans))
normalized_all = normalized_all.chunk({'lat': 50, 'lon': 50})
normalized_all = normalized_all.load()

In [ ]:
normalized_max = xr.open_dataset('max_assim_weights_wetsnow_15June4.nc')
normalized_max = normalized_max.__xarray_dataarray_variable__
normalized_max = normalized_max.where(~np.isnan(obs_nans))
normalized_max = normalized_max.chunk({'lat': 50, 'lon': 50})
normalized_max = normalized_max.load()
del obs_nans

Function to save the estimates and standard deviations 

In [8]:
def save_assimilation(ensemble, weights, title, date, var):
    #ensemble is the interpolated 5km ensemble 
    #weights are the weights produced with data assimilation 
    #title is the naem used in saving 
    #date is also used in the name
    #var is the variable that we want to produce output for 
    weights['ensemble'] = np.arange(1, 101)
    data = ensemble[var]
    data = data.load()
    print('loaded data')
    #data = mask(data)
    del ensemble
    check = xr.open_dataset('../files_for_pbs/ensemble_final1.nc')
    data['time'] = check.time
    data = data.load()
    del check
    data = data.sel(time=slice('2016-10', '2017-09'))
    data = data.chunk({'ensemble': 1, 'lat':50, 'lon':50}) 
    data = data.transpose('time', 'ensemble', 'lat', 'lon')
    #creating a weighted average and the posterior estimate 
    assim = data * weights
    ens_sum = assim.sum('ensemble')
    del assim 
    print('loading posterior')
    posterior = ens_sum.load()
    print('loaded')
    del ens_sum
    name = var
    print('writing ' + name + ' mean')
    posterior.to_netcdf(name + '_' +title + '_' + date +'_est.nc')
    print('done')
    #using the posterior estimate to create the standard deviation 
    #Calculating the standard deviation using the rule of expectations to calculate variance
    e2 =  posterior ** 2
    del posterior
    ee2 = data**2
    print('loading first')
    del data 
    ee2 = ee2 * weights
    del weights
    ee2 = ee2.sum('ensemble')
    print('ee2 done')
    std = ee2 - e2
    print('std done')
    del ee2
    del e2
    std = np.sqrt(std)
    print('loading')
    std = std.load()
    print('loading done')
    print ('writing ' + name + ' std')
    std.to_netcdf(name +'_'+title+'_'+date+'_std.nc')
    print('done')
    del std
    